We will work with the data provided by the [Open Data portal of the City of Rome](http://dati.comune.roma.it/). 
We use datasets related to registered population from other countries based on country of origin, sex and place of residence which is available for the years 2014, 2013 and 2012. In particular we will work with the following three sets available in CSV format (using semicolons as delimiter):
* [Popolazione straniera iscritta in anagrafe per municipio, sesso e cittadinanza. Al 31-12-2014](http://dati.comune.roma.it/cms/it/dettaglio_pop_e_societa.page?contentId=DTS542)
* [Popolazione straniera iscritta in anagrafe per municipio, sesso e cittadinanza. Al 31-12-2013](http://dati.comune.roma.it/cms/it/dettaglio_pop_e_societa.page?contentId=DTS725)
* [Popolazione straniera iscritta in anagrafe per municipio, sesso e cittadinanza. Al 31-12-2012](http://dati.comune.roma.it/cms/it/dettaglio_pop_e_societa.page?contentId=DTS728)

All three datasets share the same format:
- Country of Origin
- Continent
- For each Municipio of Rome (the city is devided in 15 administrative regions) a breakdown
 * Males (M)
 * Females (M)
 * Total (MF)
- The total population (over all administrative regions) divided in
 * Males (M)
 * Females (M)
 * Total (MF)
 
### Access Datasets & Store in local filesystem
 
We start by retrieving the dataset directly from the web using [urllib](https://docs.python.org/3/library/urllib.html) method from the standard python library.

We can acquire the url of the file by using our favorite browser and doing a right-click on the image "Download CSV".

In [ ]:
import urllib.request
url = 'http://dati.comune.roma.it/cms/do/jacms/Content/incrementDownload.action?contentId=DTS542&filename=popolazione_straniera_iscritta_in_anagrafe_per_municipio_e_sesso_e_cittadinanza_al_31_dicembre_20145ee4.csv'
u = urllib.request.urlopen(url)
rawdata = u.read()

We wish to store the information retrieved to a local file so that we can work with the data without the need to re-download again and again. 

So now we work a bit more with files, this time we will create a new file and _write_ data in the file. For a nice introduction to using files in python and how to read & write files, you can follow the [Introduction to Data Processing with Python](http://opentechschool.github.io/python-data-intro/core/text-files.html).

Notice that the _open_ method has a second parameter "wb" - "w" stands for _write_ and "b" stands for _binary_.

In [ ]:
localFile = open("rome-2014.csv", "wb")
localFile.write(rawdata)
localFile.close()

#### Exercise
* Download the datasets for 2013 and 2012 and save them to local files. 

### Load Datasets in CSV format

The files retrieved follows a comma-separated format. 

We will use the [CSVREADER](https://docs.python.org/2/library/csv.html) a standard python package as explained in [Reading and writing comma-separated data](http://opentechschool.github.io/python-data-intro/core/csv.html). 

We load all the contents of the file into a [list data structure](https://docs.python.org/3/tutorial/datastructures.html) named *dataset2014*.

In [ ]:
import csv
f = open("rome-2012.csv")
rawdata2014 = []
for row in csv.reader(f, delimiter=';'):
    rawdata2014.append(row)

The first 5 rows of the file contain the header rows (title, column titles). You can check out how it looks with the following command:

In [ ]:
rawdata2014[0:5]

The last 5 rows of the file contain the footer rows (grand total, some empty lines and the name of the department responsible for the generation of the dataset). You can check out how it looks with the following command:

In [ ]:
rawdata2014[-5:]

We will remove the header and footer parts and keep only the actual data into a new list named *dataset2014*.

In [ ]:
dataset2014 = rawdata2014[5:-5]

Check out how data look like in the table that we just created.

In [ ]:
dataset2014[0]

Depending on the locale used, it is usual that the CSV library will not recognise the numbers properly and load them as text (they are within single quotes). We need to do some basic reformatting to convert them to numbers. The main issue hese is that the numbers use a dot ('.') as separator for thousands rather than a comma (','). 

In [ ]:
for row in dataset2012:
    for value in range(2, len(row)):
        row[value] = int(row[value].replace('.',''))

#### Exercise
* Load the datasets for 2013 and 2012. 

**Note:** Although the datasets of the other years are the same, the have slightly different header and footer sections.

### Sorting Data

We are now in a position to [sort the data](https://docs.python.org/3/howto/sorting.html) based on the different columns. A simple invocation of the *sort* function will sort the rows of the table based on the contents of the 1st column (that is the name of the country of origin).

In [ ]:
dataset2014.sort()

To sort using another column as key we will work with [lambda functions](https://docs.python.org/3/reference/expressions.html#lambda). In the following example we will sort the data based on the total population in the 1st municipio (that is the 5th column)

In [ ]:
dataset2014.sort(key=lambda x: x[4])

To sort the data using the last column - that is the total population - we use the following code:

In [ ]:
dataset2014.sort(key=lambda x: x[len(x)-1])

#### Exercise
* Sort the data based on the total population, in descrending order (i.e., country-of-origin with largest population should appear first)

### Combine Datasets
We now wish to combine the 3 datasets into a single data structure. We will use a [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) to store the values, using as key the country of origin. 

In [ ]:
datadict={}

The value of each entry will contain a nested dictionary based on the following keys:
* 2014 - the total population for 2014 
* 2014-M - the total male population for 2014
* 2014-F - the total female population for 2014
* 2014-1 - the total population for 2014 for the 1st municipio
* 2014-1M - the total male population for 2014 for the 1st municipio
* 2014-1F - the total female population for 2014 for the 1st municipio
* ...

Notice how we concatenate the string and the number in order to construct the key for the nested dictionary using the str() function.

In [ ]:
for row in dataset2014:
    name = row[0]
    valuedict = {}
    valuedict[2014] = row[len(row) - 1]    
    valuedict["2014-F"] = row[len(row) - 2]
    valuedict["2014-M"] = row[len(row) - 3]
    for municipio in range(1,16):
        valuedict["2014-" + str(municipio) + "M"] = row[municipio*3 - 1]
        valuedict["2014-" + str(municipio) + "F"] = row[municipio*3]
        valuedict["2014-" + str(municipio)] = row[municipio*3 + 1]        
            
    datadict[name] = valuedict

Based on this structure we can directly access a specific item using the above keys. The following example will retrieve the number of females with greek origin that liv in the 7th municipio of Rome.

In [ ]:
datadict["GRECIA"]["2014-7F"]

We will now extend the dictionary to also include the values from 2013. We need to make sure that in the 2013 dataset some countries of origin may be missing (no people from that country lived in Rome during 2013).

In [ ]:
for row in dataset2013:
    name = row[0]

    valuedict = {}
    if not (name in datadict):
        # Country does not exist, initialize 2014 keys with 0 values
        valuedict[2014] = 0
        valuedict["2014-F"] = 0
        valuedict["2014-M"] = 0
        for municipio in range(1,16):
            valuedict["2014-" + str(municipio) + "M"] = 0
            valuedict["2014-" + str(municipio) + "F"] = 0
            valuedict["2014-" + str(municipio)] = 0    
    
        datadict[name] = valuedict
    else:
        valuedict = datadict[name]
    
    valuedict[2013] = row[len(row) - 1]    
    valuedict["2013-F"] = row[len(row) - 2]
    valuedict["2013-M"] = row[len(row) - 3]
    for municipio in range(1,16):
        valuedict["2013-" + str(municipio) + "M"] = row[municipio*3 - 1]
        valuedict["2013-" + str(municipio) + "F"] = row[municipio*3]
        valuedict["2013-" + str(municipio)] = row[municipio*3 + 1]    

Similarly maybe a country of origin appearing in the 2013 datase is no longer represented in 2014 (all people left Rome).

In [86]:
for key, valuedict in datadict.items():    
    if not 2013 in valuedict:
        valuedict[2013] = 0
        valuedict["2013-F"] = 0
        valuedict["2013-M"] = 0
        for municipio in range(1,16):
            valuedict["2013-" + str(municipio) + "M"] = 0
            valuedict["2013-" + str(municipio) + "F"] = 0
            valuedict["2013-" + str(municipio)] = 0             

#### Exercise
* Repeat the above procedure also for 2012.

### Statistics over Combined Datasets
Given the two-level dictionary data structure that we just constructed we are ready to do easily compute annual statistics.

In [ ]:
print("Growth from 2013 to 2014:", datadict["GRECIA"][2014] - datadict["GRECIA"][2013])
print("Growth from 2012 to 2014:", datadict["GRECIA"][2014] - datadict["GRECIA"][2012])

We will compute the growth for each country of origin and store it within the 2nd level dictionary using the following keys:
* "2014-2013-diff" - the absolute difference in population between the two years.
* "2014-2013-growth" - the growth between the two years as a percentage.

In [90]:
for (key, value) in datadict.items():
    value["2014-2013-diff"] = value[2014] - value[2013]
    if (value[2013] == 0):
        value["2014-2013-growth"] = -1
    else:
        value["2014-2013-growth"] = (value[2014] - value[2013])/ value[2013]

#### Exercise
* Repeat the above procedure also for 2012.
* Compute the statistics for all the other categories included in the datasets.

We can now directly retrieve the values without repeating any computation.

In [97]:
print("Population change from 2013 to 2014:", datadict["GRECIA"]["2014-2013-diff"])
print("Population change as percentage from 2013 to 2014:", '{0:.2f}%'.format(100 * datadict["GRECIA"]["2014-2013-growth"]))

Population change from 2013 to 2014: -145
Population change as percentage from 2013 to 2014: -17.06%


#### Exercise
* Identify country of origin with the highest increase from 2013 to 2014, from 2012 to 2014.
* Identify country of origin with the highest decrease from 2013 to 2014, from 2012 to 2014.
* Identify municipio with the highest increase (over all countries of origin) from 2013 to 2014, from 2012 to 2014.
* Identify municipio with the highest decrease (over all countries of origin) from 2013 to 2014, from 2012 to 2014.